# Signal is proportional to fraction folded

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
from util import *
from config import * 
import generate_dataset
from study_gen import study
import plots
import ipynbname


## ∆T –  Does the Tm change with reaction Temp?

### Change in Temp (w/o prediction)
- For each A/C base in ROI, plot mutation fraction vs. T
    - should be linear for some portion, then level off when hydrolysis is faster than DMS reaction


In [ ]:
print('Temperature_samples: ', temperature_samples)')
# Edit the construct name below to change the plot
construct = '3042-O-flank_1=hp1-DB'
##################################################

plot = plots.change_in_temp_mut_frac_vs_temperature(study, temperature_samples, construct)
save_plotly_fig(ipynbname.path(), '[A] Change in temperature (w o prediction)/'+construct, plot['fig'])
plot['fig'].show()
        

### Mutation rate across the family vs pred ∆G (at 37 °C)
- Curve with Predicted ∆G
for each base in ROI, plot mutation fraction vs. pred ∆G (at 37 °C)
- fit each curve to ∆∆Goffset 


In [ ]:
print('Samples with t=37c:', samples:=study.df[study.df['temperature_k']==310]['sample'].unique())
## Edit the parameters below to change the sample and family
sample = samples[0]
family = 'hp1'
#####################

plot = plots.mut_rate_across_family_vs_deltaG(study, sample, family)
save_plotly_fig(ipynbname.path(), '[B] Change in temperature (w prediction at 37)/{}/{}'.format(sample, family), plot['fig'])
plot['fig'].show()

### Change in Temp (w/ prediction at T) 
- For each base in ROI at each temp, plot mutation fraction vs. pred ∆G (at that temp) 
    - fit each curve to ∆∆Goffset
    - simple model—the offset should be the same if the algorithm is just off by some amount??


In [ ]:
savefig2(ipynbname.path(), '[C] Change in temperature (w prediction at T)')

##   ∆t – is the signal in the linear regime at the working reaction time? 
### Change in reaction time
- For each construct at each position, plot Mutation fraction vs. reaction time
    - scatter plot
    - fit linear region (for now that might just be to the first 3 points 1-5 min)--goodness of fit??




In [ ]:
print('Samples with change in reaction time:', reaction_time_samples:=study.df[study.df['reaction_time']!=0]['sample'].unique())
# Edit the construct name below to change the plot
construct = '3042-O-flank_1=hp1-DB'
##################################################

plot = plots.change_in_reaction_time(study, samples=reaction_time_samples, construct=construct)
save_plotly_fig(ipynbname.path(), '[D] Change in reaction time/'+construct, plot['fig'])
plot['fig'].show()


## ∆[DMS] -  is the signal in the linear regime at the working concentration?
### Change in [DMS]
- For each construct at each position, plot Mutation fraction vs. [DMS]
    - fit linear region (for now that might just be to the first 3 points 0.5-1% DMS)--goodness of fit??


In [40]:
print('Samples with DMS concentration:', dms_concentration_samples:=study.df[study.df['dms_concentration_mM']>0]['sample'].unique())
# Edit the construct name below to change the plot
construct = '3042-O-flank_1=hp1-DB'
##################################################

plot = plots.change_in_dms_conc(study, samples=dms_concentration_samples, construct=construct)
save_plotly_fig(ipynbname.path(), '[E] Change in [DMS]/'+construct, plot['fig'])
plot['fig'].show()
        